In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import tensorflow as tf
from scipy.stats.kde import gaussian_kde
from numpy import linspace
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-08-23 16:17:07.253 DEBU

In [2]:
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import FewshotNerBinaryClassifier, CompositeEmbedder
from src.fewshot_ner_viz_component.data_processing import *

### Configuration parameters

In [3]:
NE_TYPES = ['PERSON']
N_EXAMPLES = 10
N_TEST_SENTENCES = 100
N_SAMPLES = 10
MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
MODELS = ['svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
elmo_params = [({'custom_weights':False, 'weights':[]}, 'Default'),
               ({'custom_weights':True, 'weights':[0, 1]}, 'Last layer'),
               ({'custom_weights':True, 'weights':[1, 0]}, 'First layer'),
               ({'custom_weights':True, 'weights':[0.5, 0.5]}, 'Equaly weighted'),
               ({'custom_weights':True, 'weights':[0.25, 0.75]}, '1:3'),
               ({'custom_weights':True, 'weights':[0.75, 0.25]}, '3:1'),
               ({'custom_weights':True, 'weights':[1./3, 1./3, 1./3]}, 'Equaly weighted (accounting word emb)')]
# elmo_params = [({'custom_weights':False, 'weights':[]}, 'Default'),
#     ({'custom_weights':True, 'weights':[1./3, 1./3, 1./3]}, 'Equaly weighted (accounting word emb)')]
elmo_params = [({'custom_weights':False, 'weights':[]}, 'Default'),
#                ({'custom_weights':True, 'weights':[(1./6 + 1./3)/2, (1./2 + 1./3)/2, (1./6 + 1./3)/2]}, 'Weights from paper for SRL'),
               ({'custom_weights':True, 'weights':[0.20324296*1.1340598, 0.2170538*1.1340598, 0.46144727*1.1340598]}, 'Learned weights')]

In [6]:
# embedders = []
# for params,_ in elmo_params:
#     embedders.append(CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False, elmo_params=params))

In [7]:
def plotPDE(similarities, sim_type='cosine', info=''):
    sim_flat = np.array(flatten_sim(similarities)[sim_type])
    sim_test_words = sim_flat[y_test == 0]
    sim_test_ne = sim_flat[y_test == 1]
    plt.figure(figsize=(7,5))
    kde_words = gaussian_kde(sim_test_words)
    dist_space_words = linspace( min(sim_test_words), max(sim_test_words), 100 )
    kde_ne = gaussian_kde( sim_test_ne )
    dist_space_ne = linspace( min(sim_test_ne), max(sim_test_ne), 100 )
    plt.plot( dist_space_words, kde_words(dist_space_words), color='green', label='words' )
    plt.plot( dist_space_ne, kde_ne(dist_space_ne), color='red',  label='named entities' )
    plt.legend(loc='upper right')
    plt.title('Cosine NE similarity PDE ({})'.format(info))
    plt.grid()

In [8]:
ne_type = NE_TYPES[0]
print('NE type: {}'.format(ne_type))
dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)
# np.random.seed(313)
tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
# np.random.seed(23)
tokens_test,tags_test = get_data_sample(dataset['test'], N_TEST_SENTENCES)
y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])

print('+++++ Samples stats +++++')
print('Support set:')
print_data_props(calc_data_props(tokens_train,tags_train))
print('Tests:')
print_data_props(calc_data_props(tokens_test,tags_test))

f1_scores = []
sim_list = []
prob_list = []
pred_list = []
# print(len(embedders))
# for i, embedder in enumerate(embedders):
i = 1
for params,_ in elmo_params:
    embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False, elmo_params=params)
    print('~~~~~~~~ Embedder {} ~~~~~~~~'.format(i))
    ner_bin_clf = FewshotNerBinaryClassifier(embedder)
    ner_bin_clf.train_on_batch(tokens_train, tags_train)
    results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
    method = MODELS[0]
    f1 = f1_score(y_test, results[method]['pred'])
#     print('F1 score: {:.4f}'.format(f1))
    f1_scores.append(f1)
    sim_list.append(results[method].get('sim'))
    prob_list.append(results[method]['probas'])
    pred_list.append(results[method]['pred'])
    del embedder
    del ner_bin_clf
    i += 1

NE type: PERSON
Num of train sentences: 12195
Num of valid sentences: 1553
Num of test sentences: 1573
+++++ Samples stats +++++
Support set:
#sentences = 10, #tokens = 238, #ne = 22, #ne / #tokens = 0.092, 
Tests:
#sentences = 100, #tokens = 2540, #ne = 188, #ne / #tokens = 0.074, 
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-23 16:17:19.501 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-08-23 16:17:19.887 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

INFO:tensorflow:Initialize variable module_1/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-08-23 16:17:53.840 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module_1/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module_1/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-08-23 16:17:53.848 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module_1/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


~~~~~~~~ Embedder 2 ~~~~~~~~
Train
SVM classifier model


In [9]:
if results[method].get('sim') != None:
    for params,name in elmo_params:
        plotPDE(results[method]['sim'], sim_type='cosine', info=name)

In [10]:
print('++++++ F1 scores ++++++')
i = 0
for params,name in elmo_params:
    print('{}: {:.2f}'.format(name, f1_scores[i]*100))
    i += 1

++++++ F1 scores ++++++
Default: 83.84
Learned weights: 82.86
